<a href="https://colab.research.google.com/github/tarunku/open_llm/blob/main/Introduction_to_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Basics: Building a Simple Translation Chain

This notebook demonstrates the fundamental concepts of LangChain by building a basic translation application. We'll use the OpenAI language model (gpt-4o-mini) to translate text from English to French.

**Key Concepts:**

1. **Prompt Templates:** We define prompt templates to structure the interaction with the language model. This involves:
    - A system message that sets the context (e.g., "You are a helpful assistant...").
    - A human message that provides the input text to be translated.

2. **LLMChain:** We create an LLMChain object that links the language model and the prompt template. This enables us to easily run the translation task.

3. **Input and Output:** We format the prompt with specific input values (source and target language, text) and use `chain.run()` or `chain.invoke()` to get the translated output.

**How this shows basic LangChain object formation:**

* **Combining Prompts and LLMs:** The core of LangChain is connecting language models with prompts. This notebook showcases how to create a prompt template and link it to a language model using the `LLMChain` class.

* **Flexible Input and Output:** The `LLMChain` allows you to provide structured input (using dictionaries) and receive structured output, making it easy to integrate with other applications.

* **Building Blocks for More Complex Chains:** This simple translation example serves as a foundation for building more complex LangChain applications. You can extend this concept to create chains for various tasks like summarization, question answering, and code generation.


**In Summary:**

This notebook provides a hands-on introduction to LangChain by demonstrating how to:

- Define prompt templates.
- Create an LLMChain to link a language model and a prompt.
- Run the chain with specific input values to get the desired output.

By understanding these basic concepts, you'll be well-equipped to explore more advanced features of LangChain and build powerful language-based applications.

## Installing Dependencies

This code cell installs the required Python packages using `pip`:
**Breakdown:**

- `!pip install`: This is the command to install packages using `pip`, the Python package installer.
- `-q`: This flag makes the installation quieter, reducing the output displayed during the process.
- `-U`: This flag upgrades any existing packages to the latest versions.
- `langchain`: Installs the core LangChain library.
- `langchain-openai`: Installs the package for integration with OpenAI's models.
- `langchain_community`: Installs community-developed LangChain extensions.
- `transformers`: Installs the Hugging Face Transformers library for working with pre-trained language models.
- `torch`: Installs the PyTorch library for deep learning.
- `accelerate`: Installs the Hugging Face Accelerate library for training and inference optimizations.
- `bitsandbytes`: Installs the bitsandbytes library, often used for quantization techniques to reduce model size.

In [ ]:
!pip install -qU langchain langchain-openai langchain_community transformers torch accelerate bitsandbytes

## Importing Necessary Libraries

This section imports the required libraries for the LangChain application.

In [ ]:
import torch
from langchain import LLMChain
from huggingface_hub import login
from google.colab import userdata

from langchain.llms import HuggingFacePipeline
from langchain.chat_models import (
    ChatOpenAI,
    init_chat_model
)

from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TextStreamer,
    BitsAndBytesConfig,
    AutoModelForSpeechSeq2Seq,
    pipeline
)


In [ ]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

## Loading the Pipeline: `load_pipeline` Function

This function, `load_pipeline`, is responsible for loading and configuring a language model pipeline for text generation using Hugging Face Transformers and bitsandbytes for 4-bit quantization.

In [ ]:
def load_pipeline(READER_MODEL_NAME):
  bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
  )

  model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, device_map="auto", quantization_config=bnb_config)

  tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)
  tokenizer.pad_token = tokenizer.eos_token

  streamer = TextStreamer(tokenizer)

  return pipeline(
      model=model,
      tokenizer=tokenizer,
      task="text-generation",
      do_sample=True,
      temperature=0.2,
      repetition_penalty=1.1,
      return_full_text=False,
      max_new_tokens=1000,
      streamer=streamer,
  ), tokenizer

## Setting up Language Models

This section initializes three different language models for use with LangChain:

### 1. `first_llm`
- **Purpose:** This line initializes an instance of the `ChatOpenAI` class to interact with OpenAI's GPT-4 Turbo model.
- **Model:** It uses the `gpt-4o-mini` model.
- **Authentication:** It retrieves your OpenAI API key using `userdata.get('OPENAI_API_KEY')`. Make sure you have stored your API key using `userdata.set('OPENAI_API_KEY', 'your_api_key')`.

### 2. `second_llm`
- **Purpose:** This section also initializes an OpenAI model using `init_chat_model` with additional configurations.
- **Model:** It uses the same `gpt-4o-mini` model.
- **Temperature:** It sets the `temperature` to 0 for deterministic output.
- **Configuration:** It defines configurable fields for later modification.
- **Prefix:** It uses the `config_prefix` "first" for chains with multiple models.
- **Authentication:** It also uses your OpenAI API key for authentication.

### 3. `os_llm`
- **Purpose:** This part sets up an open-source language model using `HuggingFacePipeline`.
- **Model:** It uses the `zephyr-7b-beta` model from Hugging Face.
- **Pipeline:** It utilizes the `load_pipeline` function (defined elsewhere) to load and configure the model and its tokenizer, including 4-bit quantization and a text streamer.


**Summary:**

The code sets up three language models: two from OpenAI (`first_llm` and `second_llm`) and one open-source model (`os_llm`). You can use these models in LangChain applications by assigning them to the `llm` parameter when creating chain objects.

In [ ]:

# Set up the OpenAI model,
first_llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=userdata.get('OPENAI_API_KEY'))

# Set up the OpenAI model
second_llm = init_chat_model(
    model="gpt-4o-mini",
    openai_api_key=userdata.get('OPENAI_API_KEY'),
    temperature=0,
    configurable_fields=("model", "model_provider", "temperature", "max_tokens"),
    config_prefix="first",  # useful when you have a chain with multiple models
)

# Set up the Opensource model,
pipeline, tokenizer = load_pipeline("HuggingFaceH4/zephyr-7b-beta")
os_llm = HuggingFacePipeline(pipeline=pipeline)


## Defining the Prompt Template

This code snippet defines the prompt template that will be used to interact with the language model for translation.

**1. System Message:**
- `system_template`: This string defines the system message, which sets the context for the language model. It instructs the model to act as a helpful assistant for translation.
- `system_message_prompt`: This creates a `SystemMessagePromptTemplate` object from the `system_template`. This object represents the system message within the prompt template.
- `{input_language}` and `{output_language}`: These are placeholders that will be filled with specific language codes when the prompt is used.


**2. Human Message:**
- `human_template`: This string defines the human message, which will contain the text to be translated.
- `human_message_prompt`:  This creates a `HumanMessagePromptTemplate` object from the `human_template`. This object represents the human message within the prompt template.
- `{text}`: This is a placeholder that will be replaced with the actual text to be translated.


**3. Combining Messages into a Chat Prompt:**
- `chat_prompt`: This creates a `ChatPromptTemplate` object by combining the system message prompt and the human message prompt. This object represents the complete prompt template, including both system and human messages.

**In essence, this code defines a structured prompt template that guides the language model to perform translation tasks by providing it with context and the text to be translated.**

In [ ]:
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


# invoke

**More Flexibility**: invoke provides more control and is better suited for complex scenarios. It expects an input dictionary that directly corresponds to the variables in your prompt template. This gives you finer-grained control over how the prompt is constructed.

**Advanced Features**: It also handles callbacks and allows you to access intermediate values during the chain's execution, making it more versatile.

In [ ]:
#chain = LLMChain(llm = first_llm, prompt = chat_prompt);
#chain = LLMChain(llm = second_llm, prompt = chat_prompt);
chain = LLMChain(llm = os_llm, prompt = chat_prompt);

result = chain.invoke({"input_language": "English", "output_language": "French", "text": "I love programming."})
print(result)


# run

**Simpler Interface:** run is designed for the most straightforward use cases. You provide it with a single input string, and it returns a single output string.

**Behind the Scenes:** Under the hood, run converts your input string into an input dictionary, uses that to format the prompt template, and then calls the underlying language model with the formatted prompt. Finally, it extracts the relevant part of the language model's response and returns it.

In [ ]:
#chain = LLMChain(llm = first_llm, prompt = chat_prompt);
#chain = LLMChain(llm = second_llm, prompt = chat_prompt);
chain = LLMChain(llm = os_llm, prompt = chat_prompt);

result = chain.run({"input_language": "English", "output_language": "French", "text": "I love programming."})
print(result)
